# Implementações

Fernanda Gomes e Igor Michels

In [1]:
using LinearAlgebra;

## Forma Padrão

In [2]:
function SEF(obj, c, A, sig, b, x_cons)
    """
    obj    : string (Min or Max)
    c      : vector of costs
    A      : matrix of coeff
    sig    : constraints' signal (vector of strings)
    b      : constraints (vector)
    x_cons : vector of constraints for x vector ("<=", ">=" or "" in relation to 0)
    """
    
    if obj == "Min"
        obj = "Max";
        c = -c;
    end
    
    for i in 1:length(x_cons)
        if x_cons[i] == "<="
            A[:, i] = -A[:, i];
            c[i] = -c[i];
        elseif x_cons[i] == ""
            A = hcat(A, -A[:, i]);
            append!(c, -c[i])
            x_cons[i] = ">=";
            append!(x_cons, [">="]);
        end
    end
    
    for i in 1:length(sig)
        if sig[i] == ">="
            n_lin, n_col = size(A);
            A = hcat(A, zeros(n_lin, 1));
            A[i, n_col + 1] = -1;
            sig[i] = "=";
            append!(x_cons, [">="]);
            append!(c, [0]);
        elseif sig[i] == "<="
            n_lin, n_col = size(A);
            A = hcat(A, zeros(n_lin, 1));
            A[i, n_col + 1] = 1;
            sig[i] = "=";
            append!(x_cons, [">="]);
            append!(c, [0]);
        end
    end
    return obj, c, A, sig, b, x_cons
end;

### Teste

In [3]:
obj = "Min";
c = [-1, 2, -4];
A = [ 1  5  3
      2 -1  2
      1  2 -1];
sig = [">=", "<=", "="];
b = [5, 4, 2];
x_cons = [">=", ">=", ""];

obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons)

("Max", [1, -2, 4, -4, 0, 0], [1.0 5.0 … -1.0 0.0; 2.0 -1.0 … 0.0 1.0; 1.0 2.0 … 0.0 0.0], ["=", "=", "="], [5, 4, 2], [">=", ">=", ">=", ">=", ">=", ">="])

## Iteração Simplex

In [4]:
function iteracao_simplex(c, A, b, B, N)
    x = zeros(1, size(A, 2));
    x[B] = b;
    i = findfirst(c .> 0);
    if isnothing(i)
        return B, N, x
    end
    
    coeff = A[:, i];
    t = b ./ coeff;
    if t[t .> 0] == []
        return "Ilimitado";
    end
    
    tₒ = minimum(t[t .> 0]);
    j = findfirst(t .== tₒ)
    x[i] = tₒ;
    for k in 1:length(B)
        x[B[k]] -= coeff[k] * tₒ;
    end
    
    B[j] = i;
    N = Array(1:size(A, 2));
    N = N[N .∉ Ref(B)];
    sort!(B);
    sort!(N);
    
    return B, N, x
end;

## Forma Canônica

In [5]:
function CF(c, A, b, B = nothing, N = nothing, z = 0)
    if isnothing(B)
        n_lin, n_col = size(A);
        corte = n_col - n_lin + 1;
        B = Array(corte:n_col);
        N = Array(1:(corte - 1));
    end
    
    A_B = A[:, B];
    A_B⁻¹ = inv(A_B);
    y = transpose(A_B⁻¹) * c[B];
    z = z + transpose(y) * b;
    c = c - transpose(A) * y;
    A = A_B⁻¹ * A;
    b = A_B⁻¹ * b;
    return c, A, b, B, N, z
end;

### Teste

In [6]:
obj = "Max";
c = [3, 4, 6];
A = [ 3  5 -6
      1  3 -4
     -1  1 -1];
sig = ["=", "=", "="];
b = [4, 2, -1];
x_cons = [">=", ">=", ">="];

obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons);
CF(c, A, b)

([0.0, 0.0, 0.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], [1.0, -1.0, -1.0], [1, 2, 3], Int64[], -7.0)

## Teste de Infactibilidade

In [7]:
function infeasibility(A, b)
    n, m = size(A);
    A_aux = hcat(A, Matrix(I, n, n));
    c = - vcat(zeros(m), ones(n));
    sig = ["=" for i in 1:n];
    x_cons = [">=" for i in 1:(m + n)];
    x, z = simplex("Max", c, A_aux, sig, b, x_cons, false, false);
    if z == 0
        return true;
    else
        return false;
    end
end;

## Implementação do Simplex

In [8]:
function simplex(obj, c, A, sig, b, x_cons, verify = true, printing = true)
    obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons);
    B = nothing;
    N = nothing;
    z = 0;
    
    feasible = true;
    if verify
        feasible = infeasibility(A, b);
    end
    
    if ~feasible
        println("Problema infactível");
        return "Infactível"
    end
    
    while true
        c, A, b, B, N, z = CF(c, A, b, B, N, z);
        sol = iteracao_simplex(c, A, b, B, N)
        if sol == "Ilimitado"
            println("Problema ilimitado");
            return "Ilimitado";
        else
            B, N, x = sol;
        end
        
        if c[c .> 0] == []
            if printing
                println("Ótimo em $x, com valor z = $z");
            end
            return x, z;
        end
    end
end;

### Teste

In [9]:
obj = "Max";
c = [ 2,  3,  0,  0,  0];
A = [ 1  1  1  0  0
      2  1  0  1  0
     -1  1  0  0  1];
sig = ["=", "=", "="];
b = [6, 10, 4];
x_cons = [">=", ">=", ">=", ">=", ">="];
B = nothing;
N = nothing;
z = 0;
solution = simplex(obj, c, A, sig, b, x_cons);

Ótimo em [1.0 5.0 0.0 3.0 0.0], com valor z = 17.0


# Exercícios do livro

## Seção 2.1

### Exercício 1

#### Item a

Note que as restrições determinam um sistema com três equações e três incógnitas. Assim, se esse sistema for impossível, o problema é, automaticamente, infactível. Por outro lado, se o problema for possível e determinado, devemos analisar as restrições de $x$. Por fim, se o problema for possível e indeterminado nada se pode afirmar a priori, devendo ter que analisar melhor o problema.

Vamos, analisar o determinante de $A$:

In [10]:
A_b = [ 3  5 -6
        1  3 -4
       -1  1 -1];

det(A_b)

4.0

Como $det(A)\neq 0$ e as entradas do vetor $b$ são todas distintas, o sistema é possível e determinado, assim, temos:

In [11]:
b_b = [4, 2, -1];
x_b = A_b \ b_b;
x_b

3-element Vector{Float64}:
  1.0
 -1.0000000000000004
 -1.0000000000000002

Como a solução do sistema é $x = (1, -1, -1)$, o problema linear é infactível, pois não respeita a condição $x_1, x_2, x_3 > 0$.

#### Item b

Tomando o vetor $x = (t + 1, t + 2, 0, t)$, com $t\in \mathbb{R}_+$, temos que todas as restrições são satisfeitas, pois $x_1 + x_3 - x_4 = t + 1 + 0 - t = 1$ e $x_2 + 2x_3 - x_4 = t + 2 + 2\cdot 0 - t = 2$ e, como $t\in \mathbb{R}_+$, todas as entradas do vetor são não negativas.

Agora, perceba que a função a ser maximizada pode ser expressa por $t$, que é ilimitada.

#### Item c

Seguindo um misto das ideias dos dois itens anteriores, podemos ver que se um vetor $x$ é solução do sistema formado pelas restrições, então $x + td$, com $d = (1, 1, 1, 1, 1)$ e $t\in \mathbb{R}_+$ também será. Isso vai nos levar a uma função objetivo crescente e ilimitada (uma função linear do tipo $k + 2t$, com $k = c^\top x$). Assim, para provar que o problema é ilimitado, basta encontrar um vetor $x$ com entradas não negativas que satisfaz as restrições do sistema $Ax = b$. Tal vetor pode ser encontrado facilmente:

In [12]:
A_c = [ 4  2  1 -6 -1
       -1  1 -4  1  3
        3 -6  5  3 -5];

b_c = [11, -2, -8];
x_c = A_c \ b_c .+ 2;
x_c

5-element Vector{Float64}:
 2.06838805477644
 3.009404388714734
 2.3537920035197715
 0.6155474894131876
 1.9528680635758675

O que finaliza a prova de que o problema é ilimitado.

#### Item d

Utilizando os próprios vetores deduzidos nos itens anteriores, podemos ver que, como a função objetivo do item b é o próprio $t$, basta tomar $t = 5000$, chegando em $x_b = (5001, 5002, 0, 5000)$. Já para o item c, a função objetivo é da forma $k + 2t$, com $k = c^\top x_c$. Temos um $x_c$ inicial, assim podemos encontrar $k$ e, com isso, encontrar $t$:

In [13]:
c = [1, -2, 1, 1, 1];
k = c' * x_c;
t = (5000 - k) / 2

2499.514106583072

Assim, a solução que buscamos é $x_c + td$:

In [14]:
d = [1, 1, 1, 1, 1];
x = x_c + t * d;
x

5-element Vector{Float64}:
 2501.5824946378484
 2502.523510971787
 2501.8678985865918
 2500.129654072485
 2501.466974646648

### Exercício 2

#### (i)

Podemos tomar o vetor $y = \begin{bmatrix}\frac{9}{2} \\ -\frac{7}{3} \\ -\frac{7}{6}\end{bmatrix}$, a matriz $A = \begin{bmatrix}1 & 1 \\ 2 & 1 \\ -1 & 1\end{bmatrix}$ e o vetor $b = \begin{bmatrix}6 \\ 10 \\ 4\end{bmatrix}$ que, como já vimos anteriormente no livro, sabemos que o problema dado pela restrição $Ax \leq b$ e $x \geq 0$ possui solução. Agora, perceba que $y^\top A = \begin{bmatrix}1 \\ 1\end{bmatrix}$, enquanto $b^\top y = -1$, como o enunciado pede, logo, a afirmação é falsa.

#### (ii)

Como $y\geq 0$ podemos multiplicar ambos os lados por $y^\top$ temos
$y^\top A x \leq y^\top b = b^\top y < 0$. Mas como $v = y^\top A > 0$ e $x > 0$, então $vx > 0$, o que é um absurdo, logo, o problema não tem solução.

## Seção 2.2

### Exercício 1

#### Item a

Utilizando a função elaborada anteriormente, temos:

In [15]:
obj = "Min";
c = [2, -1, 4, 2, 4];
A = [ 1  2  4  7  3
      2  8  9  0  0
      1  1  0  2  6
     -3  4  3  1 -1];
sig = ["<=", "=", ">=", ">="];
b = [1, 2, 3, 4];
x_cons = [">=", ">=", "", ">="];

obj, c, A, sig, b, x_cons = SEF(obj, c, A, sig, b, x_cons);
println("Objetivo: $obj");
println("Vetor de custos: $c");
println("Matrix A:")
for i in 1:size(A, 1)
    println(A[i, :]);
end
println();
println("Sinais das restrições: $sig");
println("Vetor b: $b");
println("Sinais das restrições de x: $x_cons");

Objetivo: Max
Vetor de custos: [-2, 1, -4, -2, -4, 4, 0, 0, 0]
Matrix A:
[1.0, 2.0, 4.0, 7.0, 3.0, -4.0, 1.0, 0.0, 0.0]
[2.0, 8.0, 9.0, 0.0, 0.0, -9.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 0.0, 2.0, 6.0, 0.0, 0.0, -1.0, 0.0]
[-3.0, 4.0, 3.0, 1.0, -1.0, -3.0, 0.0, 0.0, -1.0]

Sinais das restrições: ["=", "=", "=", "="]
Vetor b: [1, 2, 3, 4]
Sinais das restrições de x: [">=", ">=", ">=", ">=", ">=", ">=", ">=", ">="]


#### Item b

Primeiramente, vamos mudar o problema de minimização para maximização por meio da troca da função custo de $c^\top x + d^\top y$ para $-c^\top x - d^\top y$.

Agora, sejam $n_A$ e $n_D$ as quantidades de linhas das matrizes $A$ e $D$, respectivamente. Para a restrição $Ax \geq b$ podemos subtrair $I_{n_A}z$, onde $I_{n_A}$ é a identidade de dimensão $n_A$ x $n_A$ e $z \geq 0$ é um vetor de $\mathbb{R}^{n_A}$, representando as variáveis de folga. Dessa forma, a primeira restrição ficará $Ax - I_{n_A}z = b$. Para a segunda restrição, como $y$ está livre, podemos escrever $y = y^+ - y^-$, onde $y^+, y^- \geq 0$, assim, podemos simplesmente reescrever a segunda restrição como $Bx + Dy^+ - Dy^- = f$, o que finaliza a padronização, gerando o problema:

$\max -c^\top x - d^\top y$

sujeito a:

$Ax - I_{n_A}z = b$

$Bx + Dy^+ - Dy^- = f$

$x, y^+, y^⁻, z \geq 0$

## Seção 2.3

### Exercício 1

#### Item a

Aplicando as funções elaboradas anteriormente, temos:

In [16]:
c = [-1, 0, 0, 2];
A = [-1  1  0  2
      1  0  1 -3];
b = [2, 3];
B = [2, 3];
N = [1, 4];

B, N, x = iteracao_simplex(c, A, b, B, N);
c, A, b, B, N, z = CF(c, A, b, B, N, 0);
z, x

(2.0, [0.0 0.0 6.0 1.0])

#### Item b

Analisando rapidamente o problema, vemos que o algoritmo irá escolher como valor para ir para a base a terceira coluna, pois a quarta tem coeficiente $-6$ na função objetivo. Como na matriz $A$ a terceira coluna é toda negativa, esse problema é ilimitado. Assim, aplicando o Simplex, devemos chegar a essa mesma conclusão:

In [17]:
c = [0, 0, 4, -6];
A = [ 1  0 -1  1
      0  1 -3  2];
b = [2, 1];
B = [1, 2];
N = [3, 4];

sol = iteracao_simplex(c, A, b, B, N);
sol

"Ilimitado"

## Seção 2.4

### Exercício 1

#### Item a

O enunciado está muito confuso, não conseguimos entender o que é para provar, mas parece falso, vide exemplo de c, vetor ii. A matriz $A_J$ associada tem $3$ linhas e $4$ colunas, logo, suas colunas não são LI.

#### Item b

Mesma coisa do item a. Agora o contra exemplo é o vetor i. A matriz $A_J$ associada é $3$x$2$, suas duas colunas são LI mas não resolve o sistema.

#### Item c e d

Como não temos restrições acerca de $x$, toda solução básica é uma solução básica factível, então, basta verificar se os vetores satisfazem $Ax = b$.

In [18]:
A = [ 1  1  0  2  1  1  1
      0  2  2  0  0 -2  1
      1  2  1  5  4  3  3];
b = [2, 2, 6];
i = [1, 1, 0, 0, 0, 0, 0];
ii = [2, -1, 2, 0, 1, 0, 0];
iii = [1, 0, 1, 0, 1, 0, 0];
iv = [0, 0, 1, 1, 0, 0, 0];
v = [0, 0.5, 0, 0, 0.5, 0, 1];

println(sum(A * i .== b) == 3)
println(sum(A * ii .== b) == 3)
println(sum(A * iii .== b) == 3)
println(sum(A * iv .== b) == 3)
println(sum(A * v .== b) == 3)

false
true
true
true
true


Já se considerarmos a factibilidade como sendo o fato de $x > 0$ (hipótese não apresentada no enunciado), então o item c permanece como acima e o d muda:

In [19]:
A = [ 1  1  0  2  1  1  1
      0  2  2  0  0 -2  1
      1  2  1  5  4  3  3];
b = [2, 2, 6];
i = [1, 1, 0, 0, 0, 0, 0];
ii = [2, -1, 2, 0, 1, 0, 0];
iii = [1, 0, 1, 0, 1, 0, 0];
iv = [0, 0, 1, 1, 0, 0, 0];
v = [0, 0.5, 0, 0, 0.5, 0, 1];

println((sum(A * i .== b) == 3) * (sum(i .>= 0) == 7))
println((sum(A * ii .== b) == 3) * (sum(ii .>= 0) == 7))
println((sum(A * iii .== b) == 3) * (sum(iii .>= 0) == 7))
println((sum(A * iv .== b) == 3) * (sum(iv .>= 0) == 7))
println((sum(A * v .== b) == 3) * (sum(v .>= 0) == 7))

false
false
true
true
true


### Exercício 2

#### Item a

In [20]:
c = [1, -2, 0, 1, 3];
A = [ 1 -1  2 -1  0
      2  0  1 -1  1];
b = [1, -1];
B = [1, 4];
N = [2, 3, 5];
c, A, b, B, N, z = CF(c, A, b, B, N, 0);
println("Novo vetor c: $c");
println("Novo vetor b: $b");
println("Nova matriz A:");
for i in 1:size(A, 1)
    println(A[i, :]);
end

Novo vetor c: [0.0, -5.0, 4.0, 0.0, 1.0]
Novo vetor b: [-2.0, -3.0]
Nova matriz A:
[1.0, 1.0, -1.0, 0.0, 1.0]
[0.0, 2.0, -3.0, 1.0, 1.0]


Onde vemos que a solução básica é infactível, pois o vetor $b$ possui entradas negativas, enquanto $x$ deve possuir apenas entradas não negativas.

#### Item b

In [21]:
c = [1, -2, 0, 1, 3];
A = [ 1 -1  2 -1  0
      2  0  1 -1  1];
b = [1, -1];
B = [3, 5];
N = [1, 2, 4];
c, A, b, B, N, z = CF(c, A, b, B, N, 0);
println("Novo vetor c: $c");
println("Novo vetor b: $b");
println("Nova matriz A:");
for i in 1:size(A, 1)
    println(A[i, :]);
end

Novo vetor c: [-3.5, -3.5, 0.0, 2.5, 0.0]
Novo vetor b: [0.5, -1.5]
Nova matriz A:
[0.5, -0.5, 1.0, -0.5, 0.0]
[1.5, 0.5, 0.0, -0.5, 1.0]


Por argumento análogo ao anterior, temos que a solução básica inicial é infactível.

### Exercício 5

#### Item a

Como o problema já está na forma canônica, podemos tomar como solução básica o vetor $\overline{x} = \begin{bmatrix}4 \\ 8 \\ 5 \\ 0 \\ 0 \\ 0\end{bmatrix}$.

#### Item b

Ao aplicar o Simplex devemos trocar algum valor de $B$ com algum de $N$. Porém, como temos que $c_B = 0$ e $c_N < 0$, isso nos daria um custo negativo, o que diminuiria o valor da função objetivo. Logo, estamos no ponto ótimo.

#### Item c

Como no item anterior vimos que $17$ é o valor ótimo da função objetivo, qualquer nova solução ótima deve ter valor igual a $17$. Além disso, para termos esse valor na função objetivo devemos ter, obrigatoriamente, $x_4 = x_5 = x_6 = 0$. Com essas restrições, podemos ver que $x_1$, $x_2$ e $x_3$ estão determinados e que $J = \{j : x'_j \neq 0\} \subseteq B$. Agora, podemos ver que $A_J = I$, ou seja, suas colunas são LI e, com isso, $x'$ deve ser solução básica.

#### Item d

Por argumento análogo ao do item b, temos que $\overline{x}$ é solução ótima.

#### Item e

Novamente, com argumento análogo ao do item c, bastando substituir $x_4$, $x_5$ e $x_6$ por $x_N$, podemos ver que os valores de $x_B$ estão determinados, o que implica em $x' = \overline{x}$.